<br>
<br>

***
***
# &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Replication of Pandas queries in SQL
***
***

<br>
<br>



### Here, we take the queries that are done in Pandas in `sentiment_revs.ipynb` and replicate them in SQL (sqlite).

In [2]:
import pandas as pd
from sqlalchemy import create_engine
sql = create_engine('sqlite://', echo=False)

/Users/Scott/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [3]:
# Load data for sentiment analysis
revs = pd.read_csv('br.csv',nrows=300000)
revs = revs.rename(index=str, columns={'reviewerName':'name', 'ratingsCount':'count', 'reviewerRatings':'revrat'})

# Send to an SQL database
revs.to_sql('revs_sql',con=sql)

In [4]:
def sqsq(query,out=None,con=sql):
    df = pd.read_sql_query(query,con)
    if type(out)==type(''):
        df.to_sql(out,con)
    return df

In [5]:
print('Number of rows =',sql.execute('''
    SELECT COUNT(*) 
    FROM revs_sql
''').scalar())
print('Number of unique reviewers =',sql.execute('''
    SELECT COUNT(DISTINCT name) 
    FROM revs_sql
''').scalar())
revs.head(10)

Number of rows = 300000
Number of unique reviewers = 77587


,bookID,title,author,rating,count,reviewsCount,name,revrat,review
0,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.73,22,1,Charles G,3.0,NaN
1,8,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",J.K. Rowling,4.77,34107,156,✿Katherine Elizabeth✿,5.0,NaN
2,3,Harry Potter and the Sorcerer's Stone,J.K. Rowling,4.44,4911929,77741,Lora,5.0,I'm going to keep this brief since there isn't...
3,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.54,1810404,28053,Cait (Paper Fury),5.0,"""Read Harry Potter!"" they said. ""It'll be fun!..."
4,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.47,1862749,29308,Diane ϟ [ Lestrange ],5.0,NaN
5,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.38,1936698,35055,Zoë,5.0,NaN
6,7,"The Harry Potter Collection (Harry Potter, #1-6)",J.K. Rowling,4.73,26702,909,Jen Holman,5.0,I do not own this spiffy box set of Harry Pott...
7,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.53,2000827,37005,Inge,5.0,NaN
8,8053,Charlie Oink (Easy Peasy People),Roger Hargreaves,4.33,3,0,Ashton,5.0,NaN
9,6,Harry Potter and the Goblet of Fire (Harry Pot...,J.K. Rowling,4.53,1897821,31817,Zoë,5.0,NaN


First we want to filter out duplicate reviews, which exist as shown here, and we want to filter out rows that have no review or no rating.

In [6]:
sqsq("""
SELECT *
FROM revs_sql
WHERE title='Ivanhoe' and name LIKE '%Bill%'
""")

,index,bookID,title,author,rating,count,reviewsCount,name,revrat,review
0,166,6445,Ivanhoe,Walter Scott,3.75,72149,1837,Bill Kerwin,4.0,None
1,167,6444,Ivanhoe,Walter Scott,3.75,72153,1837,Bill Kerwin,4.0,None
2,825,6440,Ivanhoe,Walter Scott,3.75,72147,1837,Bill Kerwin,4.0,None
3,826,6443,Ivanhoe,Walter Scott,3.75,72147,1837,Bill Kerwin,4.0,None
4,39340,40500,Ivanhoe,Walter Scott,3.75,72169,1838,Bill Kerwin,4.0,None
5,87614,93114,Ivanhoe,Walter Scott,3.75,72172,1838,Bill Kerwin,4.0,None


In [7]:
def cleanDF(dfIn):
    query = """
    SELECT MIN(bookID) AS bookID, title, author, rating, count, reviewsCount, name, revrat, review
    FROM revs_sql 
    WHERE (review IS NOT NULL) AND (revrat IS NOT NULL)
    GROUP BY title, name
    """
    return pd.read_sql_query(query,sql)
revs = cleanDF(revs)
revs.to_sql('revs_sql_clean',con=sql)

In [15]:
sqsq('SELECT * FROM revs_sql_clean LIMIT 3')

,index,bookID,title,author,rating,count,reviewsCount,name,revrat,review
0,0,19340,Black Beauty,Anna Sewell,3.94,197106,3468,James,3.0,Review\n \nBlack Beauty by Anna Sewell i...
1,1,239055,Grass Dancer,Susan Power,3.93,2036,120,Zanna,5.0,What I said in the secrecy of my thoughts was:...
2,2,27330,Hiroshima,John Hersey,3.94,44328,2152,Paquita Maria Sanchez,5.0,This book will:1) Make you cry. A lot. You wil...


In [ ]:
int('s')

In [8]:
sqsq('''
    SELECT * 
    FROM revs_sql_clean AS t1
    INNER JOIN (
        SELECT name
        FROM revs_sql
        GROUP BY name
        ORDER BY COUNT(name) DESC
        LIMIT 10
        ) AS t2
    WHERE t1.name==t2.name AND NOT t1.name=="BookDB"
''','revs_sql_clean_top_revs')

,index,bookID,title,author,rating,count,reviewsCount,name,revrat,review,name
0,10,76214,The Valley of Fear,Arthur Conan Doyle,3.98,21893,1153,Stephen,4.0,"Let’s face it, Sherlock Holmes is a prig. A v...",Stephen
1,12,295475,Anthony and Cleopatra,William Shakespeare,3.67,24459,865,Bill Kerwin,5.0,"This play is so good, it is not merely a maste...",Bill Kerwin
2,23,154421,The Law,Frédéric Bastiat,4.39,7920,602,Stephen,5.0,"6.0 stars. The newest member of my list of ""Al...",Stephen
3,26,240419,دنباله ی جستجو در تصوف ايران,عبدالحسین زرّین‌کوب,3.50,36,3,Ahmad Sharabiani,4.0,ادامه، شاید هم «جلد دوم» همان «جستجو در تصوف» ...,Ahmad Sharabiani
4,27,259682,لذات فلسفه,Will Durant,3.91,808,60,Ahmad Sharabiani,5.0,"The Pleasures of Philosophy, Will Durant عنوان...",Ahmad Sharabiani
5,31,119971,"""Cinema Paradiso""",Giuseppe Tornatore,4.20,157,10,Ahmad Sharabiani,4.0,"Nuovo cinema Paradiso, Giuseppe Tornatore تاری...",Ahmad Sharabiani
6,74,166082,"'Honour': Crimes, Paradigms, and Violence Agai...",Lynn Welchman,4.18,17,1,Chris,4.0,I want to smack people right now. Not because ...,Chris
7,95,181800,(جانِ شیفته (دورۀ چهارجلدی (L'âme enchantée),Romain Rolland,4.02,1109,58,Ahmad Sharabiani,5.0,"L'âme enchantée, Romain Rollandتاریخ نخستین خو...",Ahmad Sharabiani
8,97,185493,(کلیدر (دوره ۱۰ جلدی,محمود دولت‌آبادی,4.26,2322,184,Ahmad Sharabiani,5.0,"Kelidar, Mahmoud Dowlatabadi عنوان: کلیدر؛ نوی...",Ahmad Sharabiani
9,174,86755,1001 Curious Things: Ye Olde Curiosity Shop an...,Kate C. Duncan,3.75,4,1,Stephen,4.0,This book is a stellar study of a particular n...,Stephen


In [9]:
cols = 'bookID, title, author, rating, count, reviewsCount, name, revrat, review'

In [14]:
countQueries = ['''
SELECT name, COUNT(name) AS count{0}
FROM revs_sql_clean_top_revs
WHERE revrat=={0}
GROUP BY name
'''.format(i) for i in [1,3,4,5]]

sqsq('''
    SELECT 
        t0.name,
        rating AS croRatMean,
        AVG(revrat) AS revRatMean,
        count1,
        count3,
        count4,
        count5
    
    FROM revs_sql_clean_top_revs AS t0
    
    LEFT JOIN ({}) AS t1 ON t0.name=t1.name
    LEFT JOIN ({}) AS t3 ON t0.name=t3.name
    LEFT JOIN ({}) AS t4 ON t0.name=t4.name
    LEFT JOIN ({}) AS t5 ON t0.name=t5.name
    
    GROUP BY t0.name
'''.format(*countQueries))

,name,croRatMean,revRatMean,count1,count3,count4,count5
0,Ahmad Sharabiani,3.50,4.066667,NaN,104,674,167
1,Bill Kerwin,3.67,4.408163,NaN,60,199,280
2,Chris,4.18,3.886297,20.0,91,120,112
3,David,3.78,3.827652,34.0,136,211,147
4,John,3.87,3.974441,15.0,77,107,114
5,Manny,4.19,4.004167,14.0,110,202,154
6,Michael,4.05,3.923267,23.0,98,147,136
7,Sarah,4.07,3.625407,34.0,88,110,75
8,Stephen,3.98,3.970750,24.0,143,181,199
